In [2]:
from datetime import datetime
import requests

import pandas as pd
import numpy as np

In [46]:
from bokeh.plotting import figure, show, output_file, output_notebook

from bokeh.models import ColumnDataSource, CustomJS, Panel, Tabs
from bokeh.models.widgets import CheckboxGroup, Dropdown, RadioGroup, MultiSelect

from bokeh.layouts import column, row
from bokeh.palettes import viridis

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_file('app.html')
output_notebook()

Loading BokehJS ...

In [14]:
GH_STATES_DATA = pd.read_csv(r'K:\covid-19-data\us-states.csv', parse_dates=['date'])
GH_COUNTIES_DATA = pd.read_csv(r'K:\covid-19-data\us-counties.csv', parse_dates=['date'])

STATES = sorted(GH_STATES_DATA['state'].unique())

In [15]:
POP_DATA = pd.read_csv(r'K:\ACSDT5Y2018.B01003_data_with_overlays_2020-07-10T111915.csv')

In [91]:
def population(region):
    entry = POP_DATA[POP_DATA['NAME'] == region]
    if len(entry) != 0:
        return int(entry.values[0][2])
    else:
        return 1

In [17]:
TRACKING_DATA = pd.DataFrame.from_dict(requests.get(url='https://covidtracking.com/api/v1/states/daily.json').json())

TRACKING_DATA['datetime'] = [datetime.strptime(str(x), '%Y%m%d') for x in TRACKING_DATA['date']]
TRACKING_DATA['positivity'] = TRACKING_DATA['positive'] / TRACKING_DATA['totalTestResults'] * 100

In [11]:
def modify_doc(doc):
    
    def make_dataset(state_list, per_capita=False):

        by_state = {'date': [],
                    'avg_date': [],
                    'cases': [],
                    'avg_cases': [],
                    'deaths': [],
                    'avg_deaths': [],
                    'state': [],
                    'color': []}

        all_states = sorted(list(states_data['state'].unique()))
        palette = viridis(len(all_states))

        for state_name in state_list:

            subset = states_data[states_data['state'] == state_name]

            date_offset = np.timedelta64(3, 'D') + np.timedelta64(12, 'h')
            
            dates = subset['date']
            avg_dates = subset['date'] - date_offset

            by_state['date'].append(dates.values)
            by_state['avg_date'].append(avg_dates.values)

            if per_capita:
                pop = population(state_name)
                cases = subset['cases'].diff() / pop * 100000
                avg_cases = subset['cases'].diff().rolling(7).mean() / pop * 100000
                deaths = subset['deaths'].diff() / pop * 100000
                avg_deaths = subset['deaths'].diff().rolling(7).mean() / pop * 100000
            else:
                cases = subset['cases'].diff()
                avg_cases = subset['cases'].diff().rolling(7).mean()
                deaths = subset['deaths'].diff()
                avg_deaths = subset['deaths'].diff().rolling(7).mean()

            by_state['cases'].append(cases.values)
            by_state['avg_cases'].append(avg_cases.values)
            by_state['deaths'].append(deaths.values)
            by_state['avg_deaths'].append(avg_deaths.values)

            by_state['state'].append(state_name)
            by_state['color'].append(palette[all_states.index(state_name)])

        return ColumnDataSource(by_state)
    
    def make_plot(src):

        p = figure(title='COVID-19 Cases', x_axis_label='Date',
                   x_axis_type='datetime', y_axis_label='Cases per 100,000')
            
        p.multi_line(source=src, xs='avg_date', ys='avg_cases', legend_field='state', color='color', line_width=2)

        return p
    
    def update(attr, old, new):

        states_to_plot = sorted([state_selection.labels[i] for i in  state_selection.active])

        new_src = make_dataset(states_to_plot)

        src.data.update(new_src.data)
        
    state_selection = CheckboxGroup(labels=states, active = [0, 1])
    state_selection.on_change('active', update)
    
    controls = column(state_selection)
    
    initial_states = [state_selection.labels[i] for i in state_selection.active]
    
    src = make_dataset(initial_states)
    
    p = make_plot(src)
    
    layout = row(controls, p)
    doc.add_root(layout)
    
app = Application(FunctionHandler(modify_doc))

In [12]:
show(app)

In [ ]:
def modify_doc(doc):
    
    def make_dataset(state_list, per_capita=False):

        by_state = {'date': [],
                    'avg_date': [],
                    'cases': [],
                    'avg_cases': [],
                    'deaths': [],
                    'avg_deaths': [],
                    'state': [],
                    'color': []}

        all_states = sorted(list(states_data['state'].unique()))
        palette = viridis(len(all_states))

        for state_name in state_list:

            subset = states_data[states_data['state'] == state_name]

            date_offset = np.timedelta64(3, 'D') + np.timedelta64(12, 'h')
            
            dates = subset['date']
            avg_dates = subset['date'] - date_offset

            by_state['date'].append(dates.values)
            by_state['avg_date'].append(avg_dates.values)

            if per_capita:
                pop = population(state_name)
                cases = subset['cases'].diff() / pop * 100000
                avg_cases = subset['cases'].diff().rolling(7).mean() / pop * 100000
                deaths = subset['deaths'].diff() / pop * 100000
                avg_deaths = subset['deaths'].diff().rolling(7).mean() / pop * 100000
            else:
                cases = subset['cases'].diff()
                avg_cases = subset['cases'].diff().rolling(7).mean()
                deaths = subset['deaths'].diff()
                avg_deaths = subset['deaths'].diff().rolling(7).mean()

            by_state['cases'].append(cases.values)
            by_state['avg_cases'].append(avg_cases.values)
            by_state['deaths'].append(deaths.values)
            by_state['avg_deaths'].append(avg_deaths.values)

            by_state['state'].append(state_name)
            by_state['color'].append(palette[all_states.index(state_name)])

        return ColumnDataSource(by_state)
    
    def make_plot(src):

        p = figure(title='COVID-19 Cases', x_axis_label='Date',
                   x_axis_type='datetime', y_axis_label='Cases per 100,000')

        if len(src.data) == 1:
            p.vbar(source=src, x='date', top='cases', color='orange')
            
        p.multi_line(source=src, xs='avg_date', ys='avg_cases', legend='state', color='color', line_width=2)

        return p
    
    def update(attr, old, new):

        states_to_plot = [state_selection.labels[i] for i in  state_selection.active]

        new_src = make_dataset(states_to_plot)

        src.data.update(new_src.data)
        
    state_selection = CheckboxGroup(labels=states, active = [0, 1])
    state_selection.on_change('active', update)
    
    controls = WidgetBox(state_selection)
    
    initial_states = [state_selection.labels[i] for i in state_selection.active]
    
    src = make_dataset(initial_states)
    
    p = make_plot(src)
    
    layout = row(controls, p)
    doc.add_root(layout)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [99]:
class StateDisplay:
    
    def __init__(self):

        self.state_selection = MultiSelect(title='States:', options=STATES, value=['Alabama'], height=550)
        self.per_capita = RadioGroup(labels=['Total', 'Per Capita'], active=0)
        
        self.src = None
        self.p = None
    
    def make_dataset(self, state_list):

        by_state = {'date': [],
                    'avg_date': [],
                    'cases': [],
                    'avg_cases': [],
                    'deaths': [],
                    'avg_deaths': [],
                    'state': [],
                    'color': []}

        palette = viridis(len(STATES))

        for state_name in state_list:

            subset = GH_STATES_DATA[GH_STATES_DATA['state'] == state_name]

            date_offset = np.timedelta64(3, 'D') + np.timedelta64(12, 'h')
            
            dates = subset['date']
            avg_dates = subset['date'] - date_offset

            by_state['date'].append(dates.values)
            by_state['avg_date'].append(avg_dates.values)

            if self.per_capita.active == 0:
                cases = subset['cases'].diff()
                avg_cases = subset['cases'].diff().rolling(7).mean()
                deaths = subset['deaths'].diff()
                avg_deaths = subset['deaths'].diff().rolling(7).mean()
            else:
                pop = population(state_name)
                cases = subset['cases'].diff() / pop * 100000
                avg_cases = subset['cases'].diff().rolling(7).mean() / pop * 100000
                deaths = subset['deaths'].diff() / pop * 100000
                avg_deaths = subset['deaths'].diff().rolling(7).mean() / pop * 100000

            by_state['cases'].append(cases.values)
            by_state['avg_cases'].append(avg_cases.values)
            by_state['deaths'].append(deaths.values)
            by_state['avg_deaths'].append(avg_deaths.values)

            by_state['state'].append(state_name)
            by_state['color'].append(palette[STATES.index(state_name)])

        return ColumnDataSource(by_state)
    
    def make_plot(self, src):

        self.p = figure(title='COVID-19 Cases', x_axis_label='Date',
                        x_axis_type='datetime', y_axis_label='Total Cases')
            
        self.p.multi_line(source=src, xs='avg_date', ys='avg_cases', legend_field='state', color='color', line_width=2)
        
        self.p.legend.visible = False

        return self.p
    
    def update(self, attr, old, new):

        states_to_plot = sorted(self.state_selection.value)

        new_src = self.make_dataset(states_to_plot)

        if self.per_capita.active == 0:
            y_axis_label = 'Total Cases'
        else:
            y_axis_label = 'Cases per 100,000'

        self.p.yaxis.axis_label = y_axis_label

        self.src.data.update(new_src.data)
        
    def run(self, doc):

        self.state_selection.on_change('value', self.update)
    
        #self.per_capita.js_on_click(CustomJS(args=dict(p=self.p), code='p.reset.emit()'))
        self.per_capita.on_change('active', self.update)

        controls = column([self.state_selection, self.per_capita])

        initial_states = sorted(self.state_selection.value)

        self.src = self.make_dataset(initial_states)

        p = self.make_plot(self.src)

        layout = row(controls, p)
        doc.add_root(layout)

In [100]:
show(Application(FunctionHandler(StateDisplay().run)))